In [18]:
import glob
import numpy as np
a = np.zeros((5, 5))
session=0
for name in glob.glob('/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/*.dconn.nii'):
    print(name)
    np.savetxt(name+str(session)+'test.csv',a,delimiter=',')
    session=session+1

/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC10_mem_AllSessions_cortex.dconn.nii
/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC06_mem_AllSessions_cortex.dconn.nii
/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC02_mem_AllSessions_cortex.dconn.nii
/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC07_mem_AllSessions_cortex.dconn.nii
/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC05_mem_AllSessions_cortex.dconn.nii
/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC01_mem_AllSessions_cortex.dconn.nii
/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC04_mem_AllSessions_cortex.dconn.nii
/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC03_mem_AllSessions_cortex.dconn.nii


In [1]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
# Import libraries
import nibabel as nib
import numpy as np
import os 
import time
from nilearn import plotting
from sklearn.model_selection import StratifiedKFold, cross_val_score
import nilearn.decoding
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import KFold
cv = KFold(n_splits=4)
clf= RidgeClassifier()





In [2]:
data_file='/Volumes/GRATTONLAB/MSC/MSC_Dconns/Memory/MSC01_mem_AllSessions_cortex.dconn.nii'
data_file = nib.load(data_file)
task_data = data_file.get_data()
"""
rest_file='/Volumes/GRATTONLAB/MSC/MSC_Dconns/Rest/MSC01_REST_AllSessions_cortex.dconn.nii'
rest_file = nib.load(rest_file)
rest_data = rest_file.get_data()

mask_file='/Users/Alexis/Desktop/BarchSZ/bottomBrainMask.dtseries.nii'
mask_file=nib.load(mask_file)
mask=mask_file.get_data()
"""


"\nrest_file='/Volumes/GRATTONLAB/MSC/MSC_Dconns/Rest/MSC01_REST_AllSessions_cortex.dconn.nii'\nrest_file = nib.load(rest_file)\nrest_data = rest_file.get_data()\n\nmask_file='/Users/Alexis/Desktop/BarchSZ/bottomBrainMask.dtseries.nii'\nmask_file=nib.load(mask_file)\nmask=mask_file.get_data()\n"

In [6]:
t=task_data[:29706,:]

In [15]:
a,b,c,d=np.array_split(task_data,4)


In [10]:
a.shape

(19804, 59412)

In [13]:
59412/4

14853.0

In [73]:
# Surface based searchlight
# ----------------------
from nilearn import datasets, surface
from sklearn import neighbors
from nilearn.decoding.searchlight import search_light
# Fetch a coarse surface of the left hemisphere only for speed
fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage5')
hemi = 'left'

# To define the BOLD responses to be included within each searchlight "sphere"
# we define an adjacency matrix based on the inflated surface vertices such
# that nearby surfaces are concatenated within the same searchlight.

infl_mesh = fsaverage['infl_' + hemi]
coords, _ = surface.load_surf_mesh(infl_mesh)
radius = 3.
nn = neighbors.NearestNeighbors(radius=radius)
adjacency = nn.fit(coords).radius_neighbors_graph(coords).tolil()

taskSize=task_data.shape[0]
restSize=rest_data.shape[0]

t=np.ones(taskSize, dtype=int)
r=np.zeros(restSize, dtype=int)

#x=np.dstack((task_data, rest_data))
y=np.concatenate((t,r))




scores = search_light(rest_data, r, clf, adjacency, cv=cv, n_jobs=1)

2

In [70]:
def volume_from_cifti(data, axis):
    assert isinstance(axis, nib.cifti2.BrainModelAxis)
    data = data.T[axis.volume_mask]                          # Assume brainmodels axis is last, move it to front
    volmask = axis.volume_mask                               # Which indices on this axis are for voxels?
    vox_indices = tuple(axis.voxel[axis.volume_mask].T)      # ([x0, x1, ...], [y0, ...], [z0, ...])
    vol_data = np.zeros(axis.volume_shape + data.shape[1:],  # Volume + any extra dimensions
                        dtype=data.dtype)
    vol_data[vox_indices] = data                             # "Fancy indexing"
    return nib.Nifti1Image(vol_data, axis.affine)             # Add affine for spatial interpretation

In [71]:
volume_from_cifti(cifti_data, axes[1]).orthoview()

TypeError: unsupported operand type(s) for +: 'NoneType' and 'tuple'

In [55]:
def surf_data_from_cifti(data, axis, surf_name):
    assert isinstance(axis, nb.cifti2.BrainModelAxis)
    for name, data_indices, model in axis.iter_structures():  # Iterates over volumetric and surface structures
        if name == surf_name:                                 # Just looking for a surface
            data = data.T[data_indices]                       # Assume brainmodels axis is last, move it to front
            vtx_indices = model.vertex                        # Generally 1-N, except medial wall vertices
            surf_data = np.zeros((vtx_indices.max() + 1,) + data.shape[1:], dtype=data.dtype)
            surf_data[vtx_indices] = data
            return surf_data
    raise ValueError(f"No structure named {surf_name}")

In [56]:
_ = nlp.plot_surf(str(data_dir / "conte69/Conte69.L.inflated.32k_fs_LR.surf.gii"),
                  surf_data_from_cifti(cifti_data, axes[1], 'CIFTI_STRUCTURE_CORTEX_LEFT').mean(axis=1),
                  cmap='plasma')

NameError: name 'data_dir' is not defined

In [ ]:
def decompose_cifti(img):
    data = img.get_fdata(dtype=np.float32)
    brain_models = img.header.get_axis(1)  # Assume we know this
    return (volume_from_cifti(data, brain_models),
            surf_data_from_cifti(data, brain_models, "CIFTI_STRUCTURE_CORTEX_LEFT"),
            surf_data_from_cifti(data, brain_models, "CIFTI_STRUCTURE_CORTEX_RIGHT"))

In [ ]:
vol, left, right = decompose_cifti(cifti)
print(vol.shape, left.shape, right.shape)

In [8]:
scores
#rest_file.header.get_axis(0)

array([1., 1., 1., ..., 1., 1., 1.])

In [81]:
searchlight = nilearn.decoding.SearchLight(
    mask,
    radius=5.6, estimator=clf,
    n_jobs=n_jobs,
    verbose=1, cv=cv)
searchlight.fit(data_file, t)

DimensionError: Input data has incompatible dimensionality: Expected dimension is 4D and you provided a 2D image. See http://nilearn.github.io/manipulating_images/input_output.html.

In [1]:
import pandas as pd
from nilearn import datasets
from nilearn.image import new_img_like, load_img, get_data

# We fetch 2nd subject from haxby datasets (which is default)
haxby_dataset = datasets.fetch_haxby()

# print basic information on the dataset
print('Anatomical nifti image (3D) is located at: %s' % haxby_dataset.mask)
print('Functional nifti image (4D) is located at: %s' % haxby_dataset.func[0])

fmri_filename = haxby_dataset.func[0]
labels = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
y = labels['labels']
session = labels['chunks']

Anatomical nifti image (3D) is located at: /Users/Alexis/nilearn_data/haxby2001/mask.nii.gz
Functional nifti image (4D) is located at: /Users/Alexis/nilearn_data/haxby2001/subj2/bold.nii.gz


In [2]:
from nilearn.image import index_img
condition_mask = y.isin(['face', 'house'])

fmri_img = index_img(fmri_filename, condition_mask)
y, session = y[condition_mask], session[condition_mask]

In [3]:
import numpy as np

mask_img = load_img(haxby_dataset.mask)

# .astype() makes a copy.
process_mask = get_data(mask_img).astype(np.int)
picked_slice = 29
process_mask[..., (picked_slice + 1):] = 0
process_mask[..., :picked_slice] = 0
process_mask[:, 30:] = 0
process_mask_img = new_img_like(mask_img, process_mask)

In [6]:
test=fmri_img.header
print(test)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  4  40  64  64 216   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.   3.5  3.75 3.75 2.5  1.   1.   1.  ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 1645.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'FSL4.0'
aux_file        : b''
qform_code      : unknown
sform_code      : unknown
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : 0.0
qoffset_y       : 0.0
qoffset_z       : 0.0
srow_x          : [0. 0. 0. 0.

In [ ]:
# Make processing parallel
# /!\ As each thread will print its progress, n_jobs > 1 could mess up the
#     information output.
n_jobs = 1
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import KFold
cv = KFold(n_splits=4)
clf= RidgeClassifier()
import nilearn.decoding
# The radius is the one of the Searchlight sphere that will scan the volume
searchlight = nilearn.decoding.SearchLight(
    mask_img,
    radius=5.6, estimator=clf,
    n_jobs=n_jobs,
    verbose=1, cv=cv)
searchlight.fit(fmri_img, y)

In [ ]:
from nilearn import image
mean_fmri = image.mean_img(fmri_img)

from nilearn.plotting import plot_stat_map, plot_img, show
searchlight_img = new_img_like(mean_fmri, searchlight.scores_)

# Because scores are not a zero-center test statistics, we cannot use
# plot_stat_map
plot_img(searchlight_img, bg_img=mean_fmri,
         title="Searchlight", display_mode="z", cut_coords=[-9],
         vmin=.42, cmap='hot', threshold=.2, black_bg=True)

In [77]:
from nilearn import surface

test=surface.load_surf_data('/Volumes/GRATTONLAB/MSC/MSC_Dconns/Rest/MSC01_REST_AllSessions_cortex.dconn.nii')

TypeError: Data given cannot be loaded because it is not compatible with nibabel format:
Cifti2Image('/Volu...

In [1]:
import pandas as pd
df=pd.read_csv('~/Desktop/vertex_file.csv')

KeyboardInterrupt: 

In [21]:
import numpy as np

A=np.arange(16).reshape(4,4)
B=np.arange(16).reshape(4,4)
C=np.arange(16).reshape(4,4)
size=A.shape[0]
#training set 10 x 590000
ds=np.empty((2,size))
acc=np.empty((size,size))
for i in range(size):
    print(A[i])
    print(B[i])
    ds[0]=A[i]
    ds[1]=B[1]
    acc[i]=i

[0 1 2 3]
[0 1 2 3]
[4 5 6 7]
[4 5 6 7]
[ 8  9 10 11]
[ 8  9 10 11]
[12 13 14 15]
[12 13 14 15]


In [13]:
acc=np.empty([1,12])

In [15]:
acc[0][1]=1

In [22]:
acc

array([[0., 0., 0., 0.],
       [1., 1., 1., 1.],
       [2., 2., 2., 2.],
       [3., 3., 3., 3.]])

In [19]:
import reshape

mem=reshape.matFiles('/Users/Alexis/Desktop/MSC_Alexis/analysis/data/mvpa_data/mem/MSC01_parcel_corrmat.mat')

In [20]:
mem.shape

(10, 55278)

In [91]:
import numpy as np
#2 5,5 matrices 
a = np.zeros((5, 5, 2))

In [104]:
#sessions
#i=10
import time
start_time=time.time()
values=0
for i in range(2):
    a[:,:,i]=values
    values=values+1
print("--- %s seconds ---" % (time.time() - start_time))

--- 9.512901306152344e-05 seconds ---


In [116]:
a[1,:,0]=99
a[1,:,1]=88
#this will take the first row across all subs
#taskVertex=allTaskData[vertex,:,:]

In [109]:
a[:,:,1]

array([[ 1.,  1.,  1.,  1.,  1.],
       [99., 99., 99., 99., 99.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])

In [97]:
a[0][1][0]=12
a[0][2][1]=18


In [65]:
a[:,:,2]

array([[ 2.,  2., 18.],
       [ 2.,  2.,  2.]])

In [117]:
b=a[1,:,:]

In [128]:
b=b.T

In [119]:
#59412,2
#reshape 2,59412
#b.shape
b_new=np.reshape(b,(2,5))

In [122]:
b_new[1]

array([88., 99., 88., 99., 88.])

In [131]:
b.shape

(2, 5)

In [94]:
a[:,:,1]

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [98]:
a

array([[[ 0.,  1.],
        [12.,  1.],
        [ 0., 18.],
        [ 0.,  1.],
        [ 0.,  1.]],

       [[ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.]],

       [[ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.]],

       [[ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.]],

       [[ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.]]])

In [1]:

#import relevant packages
#this script will just start off with doing a simple CV calculation
from sklearn.linear_model import RidgeClassifier
import numpy as np
import os
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import itertools
from sklearn.model_selection import cross_val_score
import nibabel as nib
import glob
import time
#using for now; because dconns are all the same size
vertex_size=59412
scores=np.empty((1,vertex_size))
clf=RidgeClassifier()
#iterate through each index
#lets try a different approach
#first load all data
#will have to determine if there is a way to declare size based on files found in glob function
allTask=np.empty((vertex_size,vertex_size,10))
allRest=np.empty((vertex_size,vertex_size,10))


In [3]:
allTask.shape

(59412, 59412, 10)

In [19]:
import numpy as np
test=np.genfromtxt('~/Desktop/MSC01_sess9.dconn.nii1first.csv',delimiter=',')


OSError: ~/Desktop/MSC01_sess9.dconn.nii1first.csv not found.